In [1]:
vertices= ((-34.6251522,-58.4132406),
(-34.6208589,-58.379557),
(-34.6149255,-58.3792137),
(-34.5968403,-58.3994698),
(-34.5745109,-58.3960365),
(-34.5452476,-58.4434151),
(-34.5682916,-58.4669327),
(-34.5686057,-58.49595),
(-34.5946108,-58.505563),
(-34.6233159,-58.4879133),
(-34.6398418,-58.4599325),
(-34.6320736,-58.4448263),
(-34.6251522,-58.4132406))


# Colores de texto
BLACK = '\033[30m'
RED = '\033[31m'
GREEN = '\033[32m'
YELLOW = '\033[33m'
BLUE = '\033[34m'
MAGENTA = '\033[35m'
CYAN = '\033[36m'
WHITE = '\033[37m'
RESET = '\033[0m'

In [2]:
import numpy as np
from shapely.geometry import Polygon, Point
from sklearn.cluster import KMeans
from scipy.optimize import linear_sum_assignment
import math
import json
import requests


def generar_datos(vertices_poligono, cantidad_repas, cantidad_envios, proporcion_idVehiculo_1=0.5, random_state=None):
    """
    Genera puntos aleatorios dentro de un polígono, crea los 'viajes' y 'reservas' con los atributos especificados,
    y devuelve un JSON con la información de los viajes y reservas.

    Parámetros:
    - vertices_poligono: Lista de tuplas (longitud, latitud) que representan los vértices del polígono.
    - cantidad_repas: Número de repartidores a generar.
    - cantidad_envios: Número de envíos a generar.
    - proporcion_idVehiculo_1: Proporción de repartidores con idVehiculo = 1 (entre 0 y 1).
    - random_state: Semilla para la generación de números aleatorios (int, None o np.random.RandomState).

    Retorna:
    - result: Diccionario con las listas de envíos ('viajes') y repartidores ('reservas') en el formato especificado.
    """
    # Crear el polígono
    poligono = Polygon(vertices_poligono)

    # Crear un generador de números aleatorios
    rng = np.random.default_rng(random_state)

    # Función para generar puntos aleatorios dentro del polígono
    def generar_puntos_en_poligono(poligono, cantidad):
        minx, miny, maxx, maxy = poligono.bounds
        puntos = []
        while len(puntos) < cantidad:
            random_point = Point(rng.uniform(minx, maxx), rng.uniform(miny, maxy))
            if poligono.contains(random_point):
                puntos.append(random_point)
        return puntos

    # Generar ubicaciones aleatorias para repas y envíos
    puntos_repas = generar_puntos_en_poligono(poligono, cantidad_repas)
    puntos_envios = generar_puntos_en_poligono(poligono, cantidad_envios)

    # Generar 'viajes' con los atributos especificados
    viajes = []
    for idx, punto in enumerate(puntos_envios, start=1):
        viaje = {
            "distancia": round(float(rng.uniform(100, 4000)), 2),
            "id": idx,
            "idUsuario": 16556,
            "latitudOrigen": float(punto.x),
            "longitudOrigen": float(punto.y),
            "fechaProcesamientoODS": "0001-01-01T00:00:00",
            "idDireccion": 30635
        }
        viajes.append(viaje)

    # Generar 'reservas' con los atributos especificados
    reservas = []
    for idx, punto in enumerate(puntos_repas, start=1):
        if rng.uniform(0, 1) < proporcion_idVehiculo_1:
            idVehiculo = 1
            radioMaximoAsignacion = 8000
        else:
            idVehiculo = 4
            radioMaximoAsignacion = 3000
        reserva = {
            "idDireccion": 30635,
            "idUsuario": 23317,
            "id": idx,
            "latitud": float(punto.x),
            "longitud": float(punto.y),
            "idVehiculo": idVehiculo,
            "radioMaximoAsignacion": radioMaximoAsignacion
        }
        reservas.append(reserva)

    # Asegurar que el formato de los datos coincida con el especificado
    result = {
        "viajes": viajes,
        "reservas": reservas
    }

    json.dump(result, open("result.json", "w"))
    return result

def asignar_repas_envios(viajes, reservas, n_clusters, random_state=None):
    
    """
    Asigna repartidores a envíos utilizando clustering y minimización de distancias geográficas.

    Parámetros:
    - viajes: Lista de diccionarios con la información de los envíos.
    - reservas: Lista de diccionarios con la información de los repartidores.
    - n_clusters: Número inicial de clusters para KMeans.
    - random_state: Semilla para la generación de números aleatorios (int, None o np.random.RandomState).

    Retorna:
    - asignaciones: Lista de diccionarios con las asignaciones realizadas.
    - sobrantes_repas: Lista de IDs de repartidores no asignados.
    - sobrantes_envios: Lista de IDs de envíos no asignados.
    - total_asignaciones: Número total de asignaciones realizadas.
    - distancia_total_metros: Suma total de las distancias de todas las asignaciones en metros.
    - distancia_promedio_metros: Promedio de las distancias de las asignaciones en metros.
    - distancia_promedio_idvehiculo_1: Promedio de las distancias de asignación para idVehiculo = 1.
    - distancia_promedio_idvehiculo_4: Promedio de las distancias de asignación para idVehiculo = 4.
    """
    import numpy as np
    from sklearn.cluster import KMeans
    from scipy.optimize import linear_sum_assignment

    # Crear un generador de números aleatorios
    rng = np.random.default_rng(random_state)

    # Función para calcular la matriz de distancias Haversine
    def matriz_distancias_haversine(repas_cluster, envios_cluster):
        # Extraer coordenadas
        lat1 = np.radians(np.array([repa['latitud'] for repa in repas_cluster]))[:, np.newaxis]
        lon1 = np.radians(np.array([repa['longitud'] for repa in repas_cluster]))[:, np.newaxis]
        lat2 = np.radians(np.array([envio['latitudOrigen'] for envio in envios_cluster]))[np.newaxis, :]
        lon2 = np.radians(np.array([envio['longitudOrigen'] for envio in envios_cluster]))[np.newaxis, :]

        dlat = lat2 - lat1
        dlon = lon2 - lon1

        a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

        R = 6371000  # Radio de la Tierra en metros
        distancia = R * c
        return distancia  # Retorna la matriz de distancias en metros

    # Asignar IDs
    ids_repas = [reserva['id'] for reserva in reservas]
    ids_envios = [viaje['id'] for viaje in viajes]

    # Combinar todos los puntos para clustering
    puntos_todos = reservas + viajes
    etiquetas = ['repa'] * len(reservas) + ['envio'] * len(viajes)
    ids_todos = ids_repas + ids_envios  # IDs correspondientes a puntos_todos

    # Obtener coordenadas para clustering
    coordenadas = np.array(
        [[reserva['longitud'], reserva['latitud']] for reserva in reservas] +
        [[viaje['longitudOrigen'], viaje['latitudOrigen']] for viaje in viajes]
    )

    # Realizar clustering inicial
    n_clusters_current = min(n_clusters, len(coordenadas))
    kmeans = KMeans(n_clusters=n_clusters_current, random_state=random_state)
    clusters = kmeans.fit_predict(coordenadas)

    # Inicializar listas de asignaciones y sobrantes
    asignaciones = []
    sobrantes_repas = ids_repas.copy()
    sobrantes_envios = ids_envios.copy()

    distancia_total_metros = 0  # Para acumular la distancia total de las asignaciones en metros
    distancias_asignaciones = []  # Lista para almacenar las distancias de cada asignación

    # Listas para distancias por idVehiculo
    distancias_idvehiculo_1 = []
    distancias_idvehiculo_4 = []

    # Procesar cada cluster
    for cluster_id in range(n_clusters_current):
        # Obtener índices de puntos en el cluster actual
        indices_cluster = np.where(clusters == cluster_id)[0]

        # Separar repas y envíos en el cluster
        repas_cluster = []
        envios_cluster = []
        ids_repas_cluster = []
        ids_envios_cluster = []

        for idx in indices_cluster:
            etiqueta = etiquetas[idx]
            if etiqueta == 'repa':
                reserva = puntos_todos[idx]
                repas_cluster.append(reserva)
                ids_repas_cluster.append(reserva['id'])
            else:
                envio = puntos_todos[idx]
                envios_cluster.append(envio)
                ids_envios_cluster.append(envio['id'])

        # Si no hay repas o envíos en el cluster, continuar
        if len(repas_cluster) == 0 or len(envios_cluster) == 0:
            continue

        # Crear matriz de distancias
        matriz_distancias = matriz_distancias_haversine(repas_cluster, envios_cluster)

        # Resolver el problema de asignación utilizando el algoritmo húngaro
        filas_ind, columnas_ind = linear_sum_assignment(matriz_distancias)

        num_asignaciones_cluster = min(len(filas_ind), len(columnas_ind))

        # Acumular distancia total y realizar asignaciones
        for idx in range(num_asignaciones_cluster):
            i = filas_ind[idx]
            j = columnas_ind[idx]
            repa_id = ids_repas_cluster[i]
            envio_id = ids_envios_cluster[j]
            distancia = matriz_distancias[i, j]
            repa = repas_cluster[i]
            idVehiculo = repa['idVehiculo']
            asignaciones.append({
                "repa_id": repa_id,
                "envio_id": envio_id,
                "distancia_metros": distancia,
                "idVehiculo": idVehiculo  # Agregar idVehiculo a la asignación
            })
            distancia_total_metros += distancia
            distancias_asignaciones.append(distancia)

            # Acumular distancias por idVehiculo
            if idVehiculo == 1:
                distancias_idvehiculo_1.append(distancia)
            elif idVehiculo == 4:
                distancias_idvehiculo_4.append(distancia)

            # Remover asignados de los sobrantes
            if repa_id in sobrantes_repas:
                sobrantes_repas.remove(repa_id)
            if envio_id in sobrantes_envios:
                sobrantes_envios.remove(envio_id)

    # Si hay sobrantes, volver a clusterizar
    iteracion = 1
    asignaciones_en_iteracion = len(asignaciones)
    while sobrantes_repas and sobrantes_envios and n_clusters > 1:
        iteracion += 1

        # Si no hubo asignaciones en la iteración anterior, reducir el número de clusters
        if asignaciones_en_iteracion == 0:
            n_clusters -= 1

        asignaciones_en_iteracion = 0  # Reiniciar el contador de asignaciones en esta iteración

        # Actualizar puntos y etiquetas de sobrantes
        reservas_sobrantes = [reserva for reserva in reservas if reserva['id'] in sobrantes_repas]
        viajes_sobrantes = [viaje for viaje in viajes if viaje['id'] in sobrantes_envios]

        ids_repas_sobrantes = sobrantes_repas.copy()
        ids_envios_sobrantes = sobrantes_envios.copy()

        puntos_todos_sobrantes = reservas_sobrantes + viajes_sobrantes
        etiquetas_sobrantes = ['repa'] * len(reservas_sobrantes) + ['envio'] * len(viajes_sobrantes)
        ids_todos_sobrantes = ids_repas_sobrantes + ids_envios_sobrantes

        # Actualizar coordenadas
        coordenadas_sobrantes = np.array(
            [[reserva['longitud'], reserva['latitud']] for reserva in reservas_sobrantes] +
            [[viaje['longitudOrigen'], viaje['latitudOrigen']] for viaje in viajes_sobrantes]
        )

        # Calcular el número actual de clusters
        n_clusters_current = min(n_clusters, len(coordenadas_sobrantes))

        # Evitar clusters vacíos
        if n_clusters_current <= 0:
            break

        # Re-clusterizar sobrantes
        kmeans = KMeans(n_clusters=n_clusters_current, random_state=random_state)
        clusters_sobrantes = kmeans.fit_predict(coordenadas_sobrantes)

        # Procesar cada cluster de sobrantes
        for cluster_id in range(n_clusters_current):
            indices_cluster = np.where(clusters_sobrantes == cluster_id)[0]

            repas_cluster = []
            envios_cluster = []
            ids_repas_cluster = []
            ids_envios_cluster = []

            for idx in indices_cluster:
                etiqueta = etiquetas_sobrantes[idx]
                if etiqueta == 'repa':
                    reserva = puntos_todos_sobrantes[idx]
                    repas_cluster.append(reserva)
                    ids_repas_cluster.append(reserva['id'])
                else:
                    envio = puntos_todos_sobrantes[idx]
                    envios_cluster.append(envio)
                    ids_envios_cluster.append(envio['id'])

            if len(repas_cluster) == 0 or len(envios_cluster) == 0:
                continue

            # Crear matriz de distancias
            matriz_distancias = matriz_distancias_haversine(repas_cluster, envios_cluster)

            filas_ind, columnas_ind = linear_sum_assignment(matriz_distancias)

            num_asignaciones_cluster = min(len(filas_ind), len(columnas_ind))

            # Acumular distancia total y realizar asignaciones
            for idx in range(num_asignaciones_cluster):
                i = filas_ind[idx]
                j = columnas_ind[idx]
                repa_id = ids_repas_cluster[i]
                envio_id = ids_envios_cluster[j]
                distancia = matriz_distancias[i, j]
                repa = repas_cluster[i]
                idVehiculo = repa['idVehiculo']
                asignaciones.append({
                    "repa_id": repa_id,
                    "envio_id": envio_id,
                    "distancia_metros": distancia,
                    "idVehiculo": idVehiculo
                })
                distancia_total_metros += distancia
                distancias_asignaciones.append(distancia)

                # Acumular distancias por idVehiculo
                if idVehiculo == 1:
                    distancias_idvehiculo_1.append(distancia)
                elif idVehiculo == 4:
                    distancias_idvehiculo_4.append(distancia)

                asignaciones_en_iteracion += 1

                if repa_id in sobrantes_repas:
                    sobrantes_repas.remove(repa_id)
                if envio_id in sobrantes_envios:
                    sobrantes_envios.remove(envio_id)

    total_asignaciones = len(asignaciones)

    # Calcular la distancia promedio total
    if total_asignaciones > 0:
        distancia_promedio_metros = distancia_total_metros / total_asignaciones
    else:
        distancia_promedio_metros = 0

    # Calcular la distancia promedio para idVehiculo = 1
    if distancias_idvehiculo_1:
        distancia_promedio_idvehiculo_1 = sum(distancias_idvehiculo_1) / len(distancias_idvehiculo_1)
    else:
        distancia_promedio_idvehiculo_1 = 0

    # Calcular la distancia promedio para idVehiculo = 4
    if distancias_idvehiculo_4:
        distancia_promedio_idvehiculo_4 = sum(distancias_idvehiculo_4) / len(distancias_idvehiculo_4)
    else:
        distancia_promedio_idvehiculo_4 = 0

    return (asignaciones, sobrantes_repas, sobrantes_envios, total_asignaciones,
            round(distancia_total_metros,2), round(distancia_promedio_metros,2),
            round(distancia_promedio_idvehiculo_1,2), round(distancia_promedio_idvehiculo_4,2))

def calcular_metricas_asignacion(reservas_json, asignaciones_json):
    """
    Calcula las métricas de asignación a partir de los JSON de reservas y asignaciones.

    Parámetros:
    - reservas_json: Lista de diccionarios con la información de las reservas (repartidores).
      Cada reserva debe contener al menos 'id' y 'idVehiculo'.
    - asignaciones_json: Lista de diccionarios con las asignaciones.
      Cada asignación debe contener 'IdReserva' y 'DistanciaPickeo'.

    Retorna:
    - distancia_total: Suma total de las distancias de todas las asignaciones.
    - promedio_distancia_por_idVehiculo: Diccionario con el promedio de distancias de asignación
      para cada idVehiculo.
    """
    # Crear un diccionario para mapear idReserva a idVehiculo
    idVehiculo_por_idReserva = {}
    for reserva in reservas_json:
        idReserva = reserva['id']
        idVehiculo = reserva['idVehiculo']
        idVehiculo_por_idReserva[idReserva] = idVehiculo

    # Inicializar variables para acumular distancias
    distancia_total = 0
    distancias_por_idVehiculo = {}

    # Procesar cada asignación
    for asignacion in asignaciones_json:
        idReserva = asignacion['IdReserva']
        distanciaPickeo = asignacion['DistanciaPickeo']

        # Obtener el idVehiculo correspondiente al idReserva
        idVehiculo = idVehiculo_por_idReserva.get(idReserva)

        # Si no se encuentra el idVehiculo, continuar
        if idVehiculo is None:
            continue

        # Acumular la distancia total
        distancia_total += distanciaPickeo

        # Acumular distancias por idVehiculo
        if idVehiculo not in distancias_por_idVehiculo:
            distancias_por_idVehiculo[idVehiculo] = []
        distancias_por_idVehiculo[idVehiculo].append(distanciaPickeo)

    # Calcular el promedio de distancia por idVehiculo
    promedio_distancia_por_idVehiculo = {}
    for idVehiculo, distancias in distancias_por_idVehiculo.items():
        promedio = sum(distancias) / len(distancias)
        promedio_distancia_por_idVehiculo[idVehiculo] = round(promedio, 2)

    return round(distancia_total, 2), promedio_distancia_por_idVehiculo

In [43]:

datos200 = generar_datos(
    vertices,
    cantidad_repas=2000,
    cantidad_envios=2000,
    proporcion_idVehiculo_1=0.3,  # Por ejemplo, 70% de los repartidores con idVehiculo = 1
    random_state=42
)



### DATA DE CONTROL CON DATA CHICA

In [121]:
## DATOS DE CONTROL

DatosControl = json.load(open("Data/MiniData.json", "r"))

# Obtener las listas de 'viajes' y 'reservas'
viajes = DatosControl['viajes']
reservas = DatosControl['reservas']

# Realizar las asignaciones
asignaciones, sobrantes_repas, sobrantes_envios, total_asignaciones, distancia_total_metros,  distancia_promedio_metros,distancia_promedio_idvehiculo_1, distancia_promedio_idvehiculo_4 = asignar_repas_envios(
    viajes,
    reservas,
    n_clusters=1,
    random_state=42
)

print('asignaciones:', asignaciones )
print('distancia_total_metros:', distancia_total_metros)
print('distancia_promedio_metros:', distancia_promedio_metros)
print('distancia_promedio_idvehiculo_1:', distancia_promedio_idvehiculo_1)
print('distancia_promedio_idvehiculo_4:', distancia_promedio_idvehiculo_4)



asignaciones: [{'repa_id': 1, 'envio_id': 1, 'distancia_metros': np.float64(1116.703266226642), 'idVehiculo': 4}, {'repa_id': 2, 'envio_id': 2, 'distancia_metros': np.float64(1145.7106493540787), 'idVehiculo': 1}, {'repa_id': 3, 'envio_id': 3, 'distancia_metros': np.float64(1358.4814349297667), 'idVehiculo': 4}, {'repa_id': 4, 'envio_id': 4, 'distancia_metros': np.float64(811.4732656590642), 'idVehiculo': 4}, {'repa_id': 5, 'envio_id': 5, 'distancia_metros': np.float64(791.84231630318), 'idVehiculo': 1}]
distancia_total_metros: 5224.21
distancia_promedio_metros: 1044.84
distancia_promedio_idvehiculo_1: 968.78
distancia_promedio_idvehiculo_4: 1095.55


### PRUEBA CON MÉTODO DE ASIGNACIÓN GATO Y DATA GRANDE

In [122]:
datos = json.load(open("Data/100.json", "r"))

# Obtener las listas de 'viajes' y 'reservas'
viajes = datos['viajes']
reservas = datos['reservas']

# Realizar las asignaciones
asignaciones, sobrantes_repas, sobrantes_envios, total_asignaciones, distancia_total_metros,  distancia_promedio_metros,distancia_promedio_idvehiculo_1, distancia_promedio_idvehiculo_4 = asignar_repas_envios(
    viajes,
    reservas,
    n_clusters=1,
    random_state=42
)

print('asignaciones:', asignaciones )
print('distancia_total_metros:', distancia_total_metros)
print('distancia_promedio_metros:', distancia_promedio_metros)
print('distancia_promedio_idvehiculo_1:', distancia_promedio_idvehiculo_1)
print('distancia_promedio_idvehiculo_4:', distancia_promedio_idvehiculo_4)


asignaciones: [{'repa_id': 1, 'envio_id': 17, 'distancia_metros': np.float64(473.18317851511654), 'idVehiculo': 4}, {'repa_id': 2, 'envio_id': 61, 'distancia_metros': np.float64(143.9542137742818), 'idVehiculo': 4}, {'repa_id': 3, 'envio_id': 90, 'distancia_metros': np.float64(469.811788324101), 'idVehiculo': 4}, {'repa_id': 4, 'envio_id': 43, 'distancia_metros': np.float64(323.669245616569), 'idVehiculo': 4}, {'repa_id': 5, 'envio_id': 18, 'distancia_metros': np.float64(1097.5070815772526), 'idVehiculo': 1}, {'repa_id': 6, 'envio_id': 67, 'distancia_metros': np.float64(181.30092349925903), 'idVehiculo': 4}, {'repa_id': 7, 'envio_id': 63, 'distancia_metros': np.float64(917.3365616902743), 'idVehiculo': 4}, {'repa_id': 8, 'envio_id': 3, 'distancia_metros': np.float64(3364.827444115154), 'idVehiculo': 1}, {'repa_id': 9, 'envio_id': 5, 'distancia_metros': np.float64(827.4683507086547), 'idVehiculo': 1}, {'repa_id': 10, 'envio_id': 37, 'distancia_metros': np.float64(1454.2982328903975), 'i

### TEST CON METODO RAPIBOY

In [123]:
# Cargar JSON de reservas desde un archivo
reservas_json = json.load(open("Data/MiniData.json", "r"))
asignaciones_json = json.load(open("Data/asignaciones.json", "r"))

reservas = reservas_json['reservas']
# Llamar a la función
distancia_total, promedio_distancia_por_idVehiculo = calcular_metricas_asignacion(reservas, asignaciones_json)

print('distancia_total:', distancia_total)
print('promedio_distancia_por_idVehiculo:', promedio_distancia_por_idVehiculo)



distancia_total: 5228.8
promedio_distancia_por_idVehiculo: {4: 1096.53, 1: 969.6}


### COMPARATIVA GASTÓN - RAPIBOY

In [29]:
## GASTON
datos = json.load(open("Data/MiniData.json", "r"))

# Obtener las listas de 'viajes' y 'reservas'
viajes = datos['viajes']
reservas = datos['reservas']

print('TAMAÑO MATRIZ: ', len(viajes))

# Realizar las asignaciones
asignaciones, sobrantes_repas, sobrantes_envios, total_asignaciones, distancia_total_metros,  distancia_promedio_metros,distancia_promedio_idvehiculo_1, distancia_promedio_idvehiculo_4 = asignar_repas_envios(
    viajes,
    reservas,
    n_clusters=1,
    random_state=42
)


print('Gastón listo perra')
## RAPIBOY
# Configurar la URL y los encabezados de la solicitud
url = 'https://staging.rapiboy.com/v1/Externo/SimuladorCoeficienteODS'
headers = {
    'Content-Type': 'application/json',
    'Token': 'tu_token'  # Reemplaza 'tu_token' con tu token real
}

# Preparar los datos para enviar en la solicitud
# En tu ejemplo, mencionas que debes enviar 'datos' en el cuerpo de la solicitud
# Convertimos 'datos' a una cadena JSON
payload = json.dumps(datos)

# Hacer la solicitud POST a la API
response = requests.post(url, headers=headers, data=payload)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    # Parsear la respuesta JSON
    asignaciones_json = response.json()
else:
    print(f"Error en la solicitud: {response.status_code}")
    print("Contenido de la respuesta:")
    print(response.text)
    asignaciones_json = []

#asix = json.load(open("Data/asignaciones.json", "r"))

# Llamar a la función
distancia_total, promedio_distancia_por_idVehiculo = calcular_metricas_asignacion(reservas, asignaciones_json)


## PRINTS

# print('asignaciones:', asignaciones )
print('GASTON_QAsignaciones:', total_asignaciones)

print('----------------------------------')    

if distancia_total_metros > distancia_total:
    print(f"GASTON_distancia_total_metros: {RED}{distancia_total_metros}{RESET}")
else:
    print(f"GASTON_distancia_total_metros: {GREEN}{distancia_total_metros}{RESET}")

# print('distancia_promedio_metros:', distancia_promedio_metros)
if distancia_promedio_idvehiculo_1 > promedio_distancia_por_idVehiculo[1]:
    print(f"GASTON_distancia_promedio_idvehiculo_1: {RED}{distancia_promedio_idvehiculo_1}{RESET}")
else:
    print(f"GASTON_distancia_promedio_idvehiculo_1: {GREEN}{distancia_promedio_idvehiculo_1}{RESET}")

if distancia_promedio_idvehiculo_4 > promedio_distancia_por_idVehiculo[4]:
    print(f"GASTON_distancia_promedio_idvehiculo_4: {RED}{distancia_promedio_idvehiculo_4}{RESET}")
else:
    print(f"GASTON_distancia_promedio_idvehiculo_4: {GREEN}{distancia_promedio_idvehiculo_4}{RESET}")

print('----------------------------------')    

print('RAPIBOY_QAsignaciones:', len(asignaciones_json))

if distancia_total > distancia_total_metros:
    print(f"RAPIBOY_distancia_total: {RED}{distancia_total}{RESET}")
else:
    print(f"RAPIBOY_distancia_total: {GREEN}{distancia_total}{RESET}")

if promedio_distancia_por_idVehiculo[1] > distancia_promedio_idvehiculo_1:
    print(f"RAPIBOY_distancia_promedio_idvehiculo_1: {RED}{promedio_distancia_por_idVehiculo[1]}{RESET}")
else:
    print(f"RAPIBOY_distancia_promedio_idvehiculo_1: {GREEN}{promedio_distancia_por_idVehiculo[1]}{RESET}")

if promedio_distancia_por_idVehiculo[4] > distancia_promedio_idvehiculo_4:
    print(f"RAPIBOY_distancia_promedio_idvehiculo_4: {RED}{promedio_distancia_por_idVehiculo[4]}{RESET}")
else:
    print(f"RAPIBOY_distancia_promedio_idvehiculo_4: {GREEN}{promedio_distancia_por_idVehiculo[4]}{RESET}")



TAMAÑO MATRIZ:  5
Gastón listo perra
GASTON_QAsignaciones: 5
----------------------------------
GASTON_distancia_total_metros: 5224.21
GASTON_distancia_promedio_idvehiculo_1: 968.78
GASTON_distancia_promedio_idvehiculo_4: 1095.55
----------------------------------
RAPIBOY_QAsignaciones: 5
RAPIBOY_distancia_total: 2751.4
RAPIBOY_distancia_promedio_idvehiculo_1: 969.6
RAPIBOY_distancia_promedio_idvehiculo_4: 270.73


In [36]:
asignaciones_json,  distancia_total_metros,distancia_promedio_idvehiculo_1, distancia_promedio_idvehiculo_4 = asignar_repas_envios_simple(
    viajes,
    reservas
)

# print('asignaciones:', asignaciones )
print('distancia_total_metros:', distancia_total_metros)
# print('distancia_promedio_metros:', distancia_promedio_metros)
print('distancia_promedio_idvehiculo_1:', distancia_promedio_idvehiculo_1)
print('distancia_promedio_idvehiculo_4:', distancia_promedio_idvehiculo_4)



distancia_total_metros: 5224.21
distancia_promedio_idvehiculo_1: 968.78
distancia_promedio_idvehiculo_4: 1095.55


In [38]:
print(asignaciones_json)

[{"$id": "1", "IdViaje": 1, "IdReserva": 1, "DistanciaPickeo": 1116.7, "Coeficiente": 1116.7}, {"$id": "2", "IdViaje": 2, "IdReserva": 2, "DistanciaPickeo": 1145.71, "Coeficiente": 1145.71}, {"$id": "3", "IdViaje": 3, "IdReserva": 3, "DistanciaPickeo": 1358.48, "Coeficiente": 1358.48}, {"$id": "4", "IdViaje": 4, "IdReserva": 4, "DistanciaPickeo": 811.47, "Coeficiente": 811.47}, {"$id": "5", "IdViaje": 5, "IdReserva": 5, "DistanciaPickeo": 791.84, "Coeficiente": 791.84}]


: 

In [35]:
def asignar_repas_envios_simple(viajes, reservas):
    
    """
    Asigna repartidores a envíos utilizando minimización de distancias geográficas.

    Parámetros:
    - viajes: Lista de diccionarios con la información de los envíos.
    - reservas: Lista de diccionarios con la información de los repartidores.

    Retorna:
    - asignaciones_json: Lista de diccionarios con las asignaciones realizadas en el formato especificado.
    - distancia_total_metros: Suma total de las distancias de todas las asignaciones en metros.
    - distancia_promedio_idvehiculo_1: Promedio de las distancias de asignación para idVehiculo = 1.
    - distancia_promedio_idvehiculo_4: Promedio de las distancias de asignación para idVehiculo = 4.
    """
    import numpy as np
    from scipy.optimize import linear_sum_assignment

    # Función para calcular la matriz de distancias Haversine
    def matriz_distancias_haversine(repas, envios):
        # Extraer coordenadas
        lat1 = np.radians(np.array([repa['latitud'] for repa in repas]))[:, np.newaxis]
        lon1 = np.radians(np.array([repa['longitud'] for repa in repas]))[:, np.newaxis]
        lat2 = np.radians(np.array([envio['latitudOrigen'] for envio in envios]))[np.newaxis, :]
        lon2 = np.radians(np.array([envio['longitudOrigen'] for envio in envios]))[np.newaxis, :]

        dlat = lat2 - lat1
        dlon = lon2 - lon1

        a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

        R = 6371000  # Radio de la Tierra en metros
        distancia = R * c
        return distancia  # Retorna la matriz de distancias en metros

    # Asignar IDs
    ids_repas = [reserva['id'] for reserva in reservas]
    ids_envios = [viaje['id'] for viaje in viajes]

    # Inicializar listas de asignaciones y sobrantes
    asignaciones_json = []
    sobrantes_repas = ids_repas.copy()
    sobrantes_envios = ids_envios.copy()

    distancia_total_metros = 0  # Para acumular la distancia total de las asignaciones en metros
    distancias_asignaciones = []  # Lista para almacenar las distancias de cada asignación

    # Listas para distancias por idVehiculo
    distancias_idvehiculo_1 = []
    distancias_idvehiculo_4 = []

    # Crear matriz de distancias
    matriz_distancias = matriz_distancias_haversine(reservas, viajes)

    # Resolver el problema de asignación utilizando el algoritmo húngaro
    filas_ind, columnas_ind = linear_sum_assignment(matriz_distancias)

    num_asignaciones = min(len(filas_ind), len(columnas_ind))

    # Acumular distancia total y realizar asignaciones
    for idx in range(num_asignaciones):
        i = filas_ind[idx]
        j = columnas_ind[idx]
        repa_id = ids_repas[i]
        envio_id = ids_envios[j]
        distancia = float(matriz_distancias[i, j])  # Convertir a tipo float
        repa = reservas[i]
        idVehiculo = repa['idVehiculo']

        # Crear el JSON de asignación en el formato solicitado
        asignaciones_json.append({
            "$id": str(idx + 1),  # Usamos 'idx + 1' para asignar un ID incremental
            "IdViaje": envio_id,
            "IdReserva": repa_id,
            "DistanciaPickeo": round(distancia, 2),
            "Coeficiente": round(distancia, 2)  # El coeficiente es igual a la distancia
        })

        distancia_total_metros += distancia
        distancias_asignaciones.append(distancia)

        # Acumular distancias por idVehiculo
        if idVehiculo == 1:
            distancias_idvehiculo_1.append(distancia)
        elif idVehiculo == 4:
            distancias_idvehiculo_4.append(distancia)

        # Remover asignados de los sobrantes
        if repa_id in sobrantes_repas:
            sobrantes_repas.remove(repa_id)
        if envio_id in sobrantes_envios:
            sobrantes_envios.remove(envio_id)

    total_asignaciones = len(asignaciones_json)

    # Calcular la distancia promedio total
    if total_asignaciones > 0:
        distancia_promedio_metros = distancia_total_metros / total_asignaciones
    else:
        distancia_promedio_metros = 0

    # Calcular la distancia promedio para idVehiculo = 1
    if distancias_idvehiculo_1:
        distancia_promedio_idvehiculo_1 = sum(distancias_idvehiculo_1) / len(distancias_idvehiculo_1)
    else:
        distancia_promedio_idvehiculo_1 = 0

    # Calcular la distancia promedio para idVehiculo = 4
    if distancias_idvehiculo_4:
        distancia_promedio_idvehiculo_4 = sum(distancias_idvehiculo_4) / len(distancias_idvehiculo_4)
    else:
        distancia_promedio_idvehiculo_4 = 0

    return (json.dumps(asignaciones_json), round(distancia_total_metros, 2),
            round(distancia_promedio_idvehiculo_1, 2), round(distancia_promedio_idvehiculo_4, 2))